# Exemplo de Notebook Spark com Scala 

Este notebook tem um exemplo de aplicação Spark feita em Scala no notebook jupyter com o kernel spylon através da imagem docker jupyter/all-spark-notebook. Para mais detalhes, ver o arquivo de instalação presente neste repositório.

Esta aplicação irá gerar um modelo de floresta aleatória para prever o time vencedor em uma partida de League of Legends.
Os dados são obtidos através da API oficial da Riot Games, e foram compilados no kaggle neste [link](https://www.kaggle.com/datasets/datasnaek/league-of-legends) pelo usuário [Mitchel J](https://www.kaggle.com/datasnaek).

In [ ]:
## Importar as bibliotecas necessárias

## Definir a SparkSession

In [ ]:
// Para definir um cluster externo master,
// basta trocar "local[*]" por "spark://<host>:<port>"
// com os dados de host e port do master.
val spark = SparkSession.builder.master("local[*]").getOrCreate

## Carregar dados dos jogos

In [ ]:
val gamesDF = spark.read
  .option("header", value = true)
  .option("inferSchema", value = true)
  .csv("games.csv")
  .persist()

## Analisar schema e dados no dataframe

In [ ]:
gamesDF.printSchema

In [ ]:
gamesDF.show(5)

## Separar em dados de treino e teste

In [ ]:
val seed = 11011990
val Array(dataTrain, dataTest) = gamesDF.randomSplit(Array(0.8, 0.2), seed)

## Filtrar colunas relevantes ao modelo

In [ ]:
val columns = gamesDF.columns.filter(
  !Array("gameId", "creationTime", "seasonId", "winner").contains(_)
)

## Gerar assembler para criar vetor de features com as colunas desejadas

In [ ]:
val assembler = new VectorAssembler()
  .setInputCols(columns)
  .setOutputCol("features")

## Indexador para definir a coluna interesse

In [ ]:
val indexer = new StringIndexer()
  .setInputCol("winner")
  .setOutputCol("label")

## Definindo o classificador de Floresta Aleatória

In [ ]:
val randomForestClassifier = new RandomForestClassifier()
  .setImpurity("gini")
  .setMaxDepth(3)
  .setNumTrees(20)
  .setFeatureSubsetStrategy("auto")

## Definindo o pipeline de operações

In [ ]:
val stages = Array(assembler, indexer, randomForestClassifier)
val pipeline = new Pipeline().setStages(stages)

## Definindo o avaliador que será usado na validação cruzada

In [ ]:
val evaluator = new BinaryClassificationEvaluator()
  .setLabelCol("label")
  .setMetricName("areaUnderROC")

## Definindo a matriz de parâmetros que serão modificados ao se realizar várias predições. O validador cruzado irá escolher o conjunto de parâmetros com a melhor predição, de acordo com o classificador definido previamente.

In [ ]:
val paramGrid = new ParamGridBuilder()
  .addGrid(randomForestClassifier.maxBins, Array(25, 28, 31))
  .addGrid(randomForestClassifier.maxDepth, Array(4, 6, 8))
  .addGrid(randomForestClassifier.impurity, Array("entropy", "gini"))
  .build()

## Definindo o validador cruzado

In [ ]:
val cv = new CrossValidator()
  .setEstimator(pipeline)
  .setEvaluator(evaluator)
  .setEstimatorParamMaps(paramGrid)
  .setNumFolds(5)

## Treinando o modelo

In [ ]:
val cvModel: CrossValidatorModel = cv.fit(dataTrain)

## Avaliando a previsão

In [ ]:
val cvPredictionDf = cvModel.transform(dataTest)
cvPredictionDf.show

## Salvando o modelo para poder ser utilizado depois em novas previsões

In [ ]:
cvModel.write.overwrite().save("model/lol")


In [ ]:
cvModel.write.overwrite().save("model/lol")


## Definir a SparkSession

In [ ]:
// Para definir um cluster externo master,
// basta trocar "local[*]" por "spark://<host>:<port>"
// com os dados de host e port do master.
val spark = SparkSession.builder.master("local[*]").getOrCreate

## Carregar dados dos jogos

In [ ]:
val gamesDF = spark.read
  .option("header", value = true)
  .option("inferSchema", value = true)
  .csv("games.csv")
  .persist()

## Analisar schema e dados no dataframe

In [ ]:
gamesDF.printSchema

In [ ]:
gamesDF.show(5)

## Separar em dados de treino e teste

In [ ]:
val seed = 11011990
val Array(dataTrain, dataTest) = gamesDF.randomSplit(Array(0.8, 0.2), seed)

## Filtrar colunas relevantes ao modelo

In [ ]:
val columns = gamesDF.columns.filter(
  !Array("gameId", "creationTime", "seasonId", "winner").contains(_)
)

## Gerar assembler para criar vetor de features com as colunas desejadas

In [ ]:
val assembler = new VectorAssembler()
  .setInputCols(columns)
  .setOutputCol("features")

## Indexador para definir a coluna interesse

In [ ]:
val indexer = new StringIndexer()
  .setInputCol("winner")
  .setOutputCol("label")

## Definindo o classificador de Floresta Aleatória

In [ ]:
val randomForestClassifier = new RandomForestClassifier()
  .setImpurity("gini")
  .setMaxDepth(3)
  .setNumTrees(20)
  .setFeatureSubsetStrategy("auto")

## Definindo o pipeline de operações

In [ ]:
val stages = Array(assembler, indexer, randomForestClassifier)
val pipeline = new Pipeline().setStages(stages)

## Definindo o avaliador que será usado na validação cruzada

In [ ]:
val evaluator = new BinaryClassificationEvaluator()
  .setLabelCol("label")
  .setMetricName("areaUnderROC")

## Definindo a matriz de parâmetros que serão modificados ao se realizar várias predições. O validador cruzado irá escolher o conjunto de parâmetros com a melhor predição, de acordo com o classificador definido previamente.

In [ ]:
val paramGrid = new ParamGridBuilder()
  .addGrid(randomForestClassifier.maxBins, Array(25, 28, 31))
  .addGrid(randomForestClassifier.maxDepth, Array(4, 6, 8))
  .addGrid(randomForestClassifier.impurity, Array("entropy", "gini"))
  .build()

## Definindo o validador cruzado

In [ ]:
val cv = new CrossValidator()
  .setEstimator(pipeline)
  .setEvaluator(evaluator)
  .setEstimatorParamMaps(paramGrid)
  .setNumFolds(5)

## Treinando o modelo

In [ ]:
val cvModel: CrossValidatorModel = cv.fit(dataTrain)

## Avaliando a previsão

In [ ]:
val cvPredictionDf = cvModel.transform(dataTest)
cvPredictionDf.show

## Salvando o modelo para poder ser utilizado depois em novas previsões

In [ ]:
cvModel.write.overwrite().save("model/lol")


In [18]:
cvModel.write.overwrite().save("model/lol")